<a href="https://colab.research.google.com/github/DominicVerschoor/CBS-Statistical-Assistant/blob/Text2Sql/Text2SQL_T5%26premai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece
!pip install torch

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load schema from Google Drive
schema_path = '/content/drive/My Drive/schema.sql'  # Replace with the actual path
with open(schema_path, 'r') as schema_file:
    schema = schema_file.read()

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "cssupport/t5-small-awesome-text-to-sql"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [3]:
# Define your query
query = "Find the first names of all the assistant professors"

# Create the prompt
prompt = f"Schema: {schema}\nQuestion: {query}\nWrite the SQL query to answer the question:"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)

# Generate SQL
outputs = model.generate(inputs.input_ids, max_length=256, num_beams=3)

# Decode the output
generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated SQL Query:")
print(generated_sql)


Generated SQL Query:
SELECT DISTINCT FacID, MIN(FacID) FROM Faculty WHERE Faculty.Professor = 'AssocProf' AND 'Prof' = 'AssocProf'


In [5]:
!pip install -U premsql


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import torch
from premsql.executors import SQLiteExecutor
from premsql.generators import Text2SQLGeneratorHF
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paths to your database and schema
database_path = "/content/drive/My Drive/activity_1.sqlite"
schema_path = "/content/drive/My Drive/schema.sql"

# Ensure database file exists
if not os.path.exists(database_path):
    raise FileNotFoundError("Database not found. Please check the path.")

# Ensure schema file exists
if not os.path.exists(schema_path):
    raise FileNotFoundError("Schema file not found. Please check the path.")

# Load the schema from the file
with open(schema_path, "r") as schema_file:
    schema = schema_file.read()

# Initialize the SQLite executor with the database path
executor = SQLiteExecutor()

# Initialize the Text2SQL generator
generator = Text2SQLGeneratorHF(
    model_or_name_or_path="premai-io/prem-1B-SQL",  # Prem-1B-SQL model
    experiment_name="database_query_test",
    device="cuda:0" if torch.cuda.is_available() else "cpu",
    type="test"
)



2025-01-09 13:26:33,233 - [MLX-GENERATOR] - ERROR - Install mlx using: pip install mlx mlx-lm
ERROR:[MLX-GENERATOR]:Install mlx using: pip install mlx mlx-lm
2025-01-09 13:26:33,241 - [OLLAMA-GENERATOR] - WARNING - Ensure ollama is installed
2025-01-09 13:26:33,245 - [OLLAMA-GENERATOR] - WARNING - Install Ollama: curl -fsSL https://ollama.com/install.sh | sh
2025-01-09 13:26:33,248 - [OLLAMA-GENERATOR] - WARNING - Install Ollama python: pip install ollama


Mounted at /content/drive


2025-01-09 13:26:56,063 - [GENERATOR] - INFO - Created new experiment folder: /root/.cache/premsql/experiments/test/database_query_test
INFO:[GENERATOR]:Created new experiment folder: /root/.cache/premsql/experiments/test/database_query_test
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.49k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

In [21]:
# Define a natural language query
natural_language_query = (
    "Using the provided schema to understand the structure of the database, tell me the first name of the students which are male. Provide an accurate SQL query "
)

# Prepare the data_blob with enhanced prompt
data_blob = {
    "database": database_path,
    "schema": schema,
    "prompt": natural_language_query,
}

# Tokenize the input prompt for attention mask
tokenized_input = generator.tokenizer(
    data_blob["prompt"], padding=True, truncation=True, return_tensors="pt"
).to("cuda" if torch.cuda.is_available() else "cpu")

# Ensure pad_token_id and eos_token_id are distinct
if generator.tokenizer.pad_token_id is None:
    generator.tokenizer.pad_token_id = generator.tokenizer.eos_token_id + 1  # Set a unique pad token if not defined

# Generate the query with retries using the schema and improved parameters
response = generator.generate(
    data_blob=data_blob,  # Provide the data_blob with schema and prompt
    max_new_tokens=256,  # Allow for more comprehensive output
    use_executor=True,
    do_sample=True,
    temperature=0.1,  # Increase temperature slightly for more exploratory outputs
    executor=executor,  # Link the SQLite executor
    max_retries=5,
    attention_mask=tokenized_input["attention_mask"],  # Use tokenized attention mask
    pad_token_id=generator.tokenizer.pad_token_id,
    eos_token_id=generator.tokenizer.eos_token_id,
)

# Handle the response
if isinstance(response, dict) and response.get("sql_output_dataframe"):
    output = response["sql_output_dataframe"]
    if output and output["data"]:
        print("Query executed successfully!")
        print("Resulting DataFrame:")
        print(output["data"])
    else:
        print("Query executed but returned no results.")
elif isinstance(response, str):
    print("Generated SQL Query:")
    print(response)
else:
    print("Unexpected response format:", response)


Generated SQL Query:
SELECT DISTINCT first_name FROM students WHERE gender = 'Male';

This query will return the first name of the students who are male. The DISTINCT keyword is used to ensure that the result set does not contain any duplicate rows.
